# Grid Stabilization Demo

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
import fd_tools as fd

## Introduction

This notebook demonstrates the grid stabilization technique presented in [this paper](https://doi.org/10.1016/j.jcp.2006.09.017) that can stabilize high-order advection solvers by adding a few off-grid points near boundaries.

Once again, here is the code for creating a first derivative approximation.  Notice that it doesn't assume uniform grid spacing and, therefore, can work with the augmented grids from the paper with no modifications.

In [ ]:
def first_deriv_matrix(x, w):
    
    I, J, data = [], [], []
    n = len(x)
    
    # bias to the right until we can center
    p = (2*w+1)
    for i in range(w):
        I.extend(p*[i])
        J.extend(range(p))
        data.extend(fd.fd_coeff(1, x[i], x[:p]))
        
    # now fill in centered differences
    for i in range(w, n-w-1):
        I.extend(p*[i])
        J.extend(range(i-w, i+w+1))
        data.extend(fd.fd_coeff(1, x[i], x[i-w:i+w+1]))

    # bias to the left when we can't center anymore
    for i in range(n-w-1, n):
        I.extend(p*[i])
        J.extend(range(n-p, n))
        data.extend(fd.fd_coeff(1, x[i], x[n-p:]))
        
    return sp.csr_matrix((data, (I, J)))

Here I've implemented the grid stabilizations presented in the paper for $q$ up to 5.  (The stencils will have width $2q-1$ and be as centered as possible near the boundary.

In [ ]:
def stab_grid(xl, xu, nx, q):
    
    z_specs = [[], [], [0.21], [0.19], [0.13, 0.97]]
    
    if q > 5:
        raise RuntimeError('Stabilized grid for q = {} not yet implemented'.format(q))
    
    # number of points on the uniform grid
    z = z_specs[q-1]
    n_unif = nx - 2*len(z)
    dx = (xu-xl)/(n_unif-1)
    
    # add extra points and sort
    x0 = sorted(list(range(n_unif)) + z + [n_unif-1-v for v in z])
    
    # scaled to desired domain
    return xl + np.array(x0)*dx, dx


Here is a demo showing the various grids for $q=1, 2, \ldots, 5$ (bottom to top).

In [ ]:
def stab_grid_demo():
    plt.figure(figsize=(20,3))
    for q in range(1,6):
        x, _ = stab_grid(-1.8, 2.3, 15, q)
        plt.plot(x, 0*x+q, 'o')
stab_grid_demo()

## Absolute stability

Here are some absolute stability studies with the mofidied grids.  If you compare them with the second notebook, you should see why the modified grids allow for stability in the higher-order solvers.

In [ ]:
def plot_eigenvalues(a, nx, cfl, width):

    x, dx = stab_grid(0, 1, nx, width)
    
    dt = cfl*(dx/a)

    D = first_deriv_matrix(x, width).toarray()

    D = D[1:,1:]

    scaled_eigs = -a*dt*np.linalg.eigvals(D)

    plt.figure(figsize=(10,5))
    for i in range(2):
        plt.subplot(1, 2, i+1)
        plt.plot(np.real(scaled_eigs), np.imag(scaled_eigs), '.')
        theta = np.linspace(0, 2*np.pi, 1000)
        plt.plot(np.cos(theta)-1, np.sin(theta), '--')
        if i == 0:
            plt.title('Forward Euler')
            plt.axis('equal')
        else:
            plt.xlim([-0.01, 0.01])

    plt.figure(figsize=(10,5))
    for i in range(2):
        plt.subplot(1, 2, i+1)
        plt.plot(np.real(scaled_eigs), np.imag(scaled_eigs), '.')
        xf = np.linspace(-2, 0.1, 200)
        yf = np.linspace(-2, 2, 400)
        Xf, Yf = np.meshgrid(xf, yf)
        Zf = Xf + 1j*Yf
        R = 1 + Zf + Zf**2/2
        mod_R = np.abs(R)
        plt.contour(xf, yf, mod_R, levels=[1], linestyles='dashed')
        if i == 0:
            plt.title('RK2')
            plt.axis('equal')
        else:
            plt.xlim([-0.01, 0.01])

    plt.figure(figsize=(10,5))
    for i in range(2):
        plt.subplot(1, 2, i+1)
        plt.plot(np.real(scaled_eigs), np.imag(scaled_eigs), '.')
        xf = np.linspace(-3, 0.3, 300)
        yf = np.linspace(-3, 3, 600)
        Xf, Yf = np.meshgrid(xf, yf)
        Zf = Xf + 1j*Yf
        R = 1 + Zf + Zf**2/2 + Zf**3/6 + Zf**4/24
        mod_R = np.abs(R)
        plt.contour(xf, yf, mod_R, levels=[1], linestyles='dashed')
        if i == 0:
            plt.title('RK4')
            plt.axis('equal')
        else:
            plt.xlim([-0.01, 0.01])

In [ ]:
plot_eigenvalues(1.8, 50, 1.5, 3)

In [ ]:
plot_eigenvalues(1.8, 50, 1.5, 4)

In [ ]:
plot_eigenvalues(1.8, 50, 0.9, 5)

## RK4 Solver

Below is an RK4 solver for the advection problem using stabilized grids.  In this notebook, I got lazy and didn't implement the boundary condition corrections found in the third notebook.  As described in the paper linked from that notebook, a side-effect is that the CFL number will be smaller.  The point for this notebook is that the stabilization works.

In [ ]:
def solve_advection_RK4(a, D, nx, dx, g, tf, cfl, n_ret):
    
    t_ret = []
    u_ret = []
    
    # target step size
    dt = cfl*(dx/a)

    # number of time steps
    nt = int(np.ceil(tf/dt))
    
    # fix time step
    dt = tf/nt
    
    # initial data
    u = np.zeros(nx)

    t_ret.append(0)
    u_ret.append(u.copy())
    
    ret_every = int(nt/n_ret)
    
    # iterate
    for it in range(1,nt+1):
        
        k1 = -dt*a*D*u
        k2 = -dt*a*D*(u + k1/2)
        k3 = -dt*a*D*(u + k2/2)
        k4 = -dt*a*D*(u + k3)
        u += (k1 + 2*k2 + 2*k3 + k4)/6
        
        t = it*dt
        u[0] = g(t)

        if it%ret_every == 0:
            t_ret.append(t)
            u_ret.append(u.copy())
        
    return t_ret, u_ret

def boundary(t):
    return np.exp(-10*(t-2)**6)*np.cos(6*np.pi*t)

def boundary_demo():
    t = np.linspace(0, 4, 1000)
    plt.plot(t, boundary(t))
boundary_demo()


In [ ]:
def solver_demo(nx, width):
    a = 1.8
    xl, xu = 0, 18
    tf = 20
    
    # I didn't try very hard to tune these, they could 
    # be improved by fixing the RK4 boundary conditions
    cfl_specs = [1.4, 1.15, 0.3, 0.25, 0.175]
    
    if width > 5:
        raise RuntimeError('Stabilized grid for width = {} not implemented'.format(width))

    cfl = cfl_specs[width-1]

    x, dx = stab_grid(xl, xu, nx, width)
    
    D = first_deriv_matrix(x, width)
    T, U = solve_advection_RK4(a, D, nx, dx, boundary, tf, cfl, 10)

    for t,u in zip(T, U):
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.plot(x, boundary(t - x/a), color='#aaaaaa')
        plt.plot(x, u)
        plt.title('t = {:.3f}'.format(t))
        plt.ylim([-1,1])
        plt.subplot(1, 2, 2)
        plt.plot(x, u -  boundary(t - x/a))
        plt.title('t = {:.3f}'.format(t))

In [ ]:
solver_demo(400, 5)